# HS2 Tweets
### Importing Packages

In [222]:
import tweepy as tw
import pprint
import numpy as np
import nltk
import bokeh
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from collections import Counter

from wordcloud import WordCloud

from bokeh.io import output_notebook, show, reset_output
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.palettes import Plasma256
from bokeh.models import Range1d

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize

from textblob import Word, TextBlob

### Twitter API Access & Set-Up

In [223]:
file = open(r"C:\Users\yeungf8452\OneDrive - ARCADIS\HS2 (Data Science)\HS2 Twitter\twitter_api.txt", "r").readlines()

api = file[0].strip('\n')
api_secret = file[1].strip('\n')
bearer_token = file[2].strip('\n')
access_token = file[3].strip('\n')
access_token_secret = file[4].strip('\n')

In [224]:
auth = tw.OAuthHandler(api, api_secret)
auth.set_access_token(access_token, access_token_secret)
api_app = tw.API(auth, wait_on_rate_limit = True)

### Query

In [460]:
client = tw.Client(bearer_token)

hashtag = "HS2 OR hs2 lang:en"
tweet_fields=['context_annotations', 'created_at', 'entities', 'geo', 'public_metrics']
user_fields = ['location', 'public_metrics']
# user fields: https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
expansions=['geo.place_id', 'author_id']
place_fields=['name', 'geo']
max_results = 10
# place fields: https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/place

# For search options: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
# Only can use operators that are "core"
# More examples: https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md

# Use pagination if need more than 100 tweets
# query = tw.Paginator(client.search_recent_tweets, "HS2", max_results=10, limit=3)
# Use .flatten when looking through data if using paginator
# https://docs.tweepy.org/en/stable/pagination.html
# https://developer.twitter.com/en/docs/twitter-api/pagination

query = client.search_recent_tweets(query=hashtag, tweet_fields = tweet_fields, user_fields = user_fields, expansions = expansions, place_fields = place_fields, max_results=max_results)

# fields are necessary in queries to get details

In [461]:
tweets = [{'Author ID:' : tweet.author_id, 'Tweet:' : tweet.text, 'Time Created': tweet.created_at, 'Tweet_ID': tweet.id, 'Auto Context Annotations':tweet.context_annotations, 'Entities': tweet.entities, 
'Location':tweet.geo, 'Metrics':tweet.public_metrics} for tweet in query.data]

tweets_df = pd.DataFrame(tweets)

In [467]:
tweets_df_copy = tweets_df.copy()

In [403]:
tweets_df['Auto Context Annotations'][3]

[{'domain': {'id': '10',
   'name': 'Person',
   'description': 'Named people in the world like Nelson Mandela'},
  'entity': {'id': '1070735198331912192',
   'name': 'Jim McMahon',
   'description': 'British Labour Party politician'}},
 {'domain': {'id': '35',
   'name': 'Politician',
   'description': 'Politicians in the world, like Joe Biden'},
  'entity': {'id': '1070735198331912192',
   'name': 'Jim McMahon',
   'description': 'British Labour Party politician'}}]

In [369]:
# make sure it is a deep copy
tweets_df = pd.DataFrame(tweets, copy = True)
tweets_df['Auto Context Annotations Entity'] = np.empty((len(tweets_df), 0)).tolist()

for k, i in enumerate(tweets_df['Auto Context Annotations']):
    for j in range(len(i)):
        #print(i[j]['entity'])
        entity = i[j]['entity']['name']
        print(entity)
        tweets_df.loc[k, 'Auto Context Annotations Entity'].append(entity)
        #entities.append(i[j]['entity']['name'])
    #tweets_df.loc[k, 'Auto Context Annotations'] = entities

Andy Burnham
Andy Burnham


TypeError: string indices must be integers

In [347]:
tweets_df

,Author ID:,Tweet:,Time Created,Tweet_ID,Auto Context Annotations,Entities,Location,Metrics
0,507795477,@bucks_herald Along with HS2,2021-10-21 16:26:29+00:00,1451223372378091532,[],"{'mentions': [{'start': 0, 'end': 13, 'usernam...",None,"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
1,1162278542891053057,RT @RHarrabin: It is remarkable they are willi...,2021-10-21 16:26:17+00:00,1451223322189049869,[],"{'mentions': [{'start': 3, 'end': 13, 'usernam...",None,"{'retweet_count': 67, 'reply_count': 0, 'like_..."
2,910229480328892416,RT @OwsWills: @AndyBurnhamGM For what it's wor...,2021-10-21 16:24:56+00:00,1451222985587793927,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 3, 'end': 12, 'usernam...",None,"{'retweet_count': 1, 'reply_count': 0, 'like_c..."
3,910229480328892416,RT @OwsWills: @AndyBurnhamGM See this image? T...,2021-10-21 16:24:40+00:00,1451222918441144320,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 3, 'end': 12, 'usernam...",None,"{'retweet_count': 2, 'reply_count': 0, 'like_c..."
4,1667387035,RT @mcahs2: No blank cheque for #HS2 they said...,2021-10-21 16:24:03+00:00,1451222763373477897,[],"{'mentions': [{'start': 3, 'end': 10, 'usernam...",None,"{'retweet_count': 12, 'reply_count': 0, 'like_..."
5,317950583,RT @OwsWills: @AndyBurnhamGM See this image? T...,2021-10-21 16:23:06+00:00,1451222521676771340,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 3, 'end': 12, 'usernam...",None,"{'retweet_count': 2, 'reply_count': 0, 'like_c..."
6,754369335993524225,RT @ITVCentral: HS2 protests cost high-speed r...,2021-10-21 16:22:46+00:00,1451222438084218886,[],"{'urls': [{'start': 101, 'end': 124, 'url': 'h...",None,"{'retweet_count': 3, 'reply_count': 0, 'like_c..."
7,2267795582,RT @EKsaysYES: BBC\nthe news they forgot\n\nBr...,2021-10-21 16:22:19+00:00,1451222325404131330,"[{'domain': {'id': '45', 'name': 'Brand Vertic...","{'mentions': [{'start': 3, 'end': 13, 'usernam...",None,"{'retweet_count': 221, 'reply_count': 0, 'like..."
8,1074657601214910464,RT @AJPhillipsEsq: @mcahs2 General Irrelevance...,2021-10-21 16:22:14+00:00,1451222305942691849,[],"{'mentions': [{'start': 3, 'end': 17, 'usernam...",None,"{'retweet_count': 3, 'reply_count': 0, 'like_c..."
9,585857264,RT @EKsaysYES: BBC\nthe news they forgot\n\nBr...,2021-10-21 16:22:13+00:00,1451222299563208706,"[{'domain': {'id': '45', 'name': 'Brand Vertic...","{'mentions': [{'start': 3, 'end': 13, 'usernam...",None,"{'retweet_count': 221, 'reply_count': 0, 'like..."


In [270]:
np.empty((0,0)).tolist()

[]

In [228]:
user_geo = {}
for i in ['users','places']:
    try:
        parameter = query.includes[i]
        user_geo[i] = {u for u in parameter}
    except KeyError:
        continue

# since we cannot extract geographical location yet
users_df = pd.DataFrame(user_geo['users'])


[{'domain': {'id': '47',
   'name': 'Brand',
   'description': 'Brands and Companies'},
  'entity': {'id': '968089839210872833',
   'name': 'Heathrow Airport',
   'description': 'Heathrow Airport'}},
 {'domain': {'id': '65',
   'name': 'Interests and Hobbies Vertical',
   'description': 'Top level interests and hobbies groupings, like Food or Travel'},
  'entity': {'id': '839159814991167489',
   'name': 'Travel',
   'description': 'Travel'}},
 'Heathrow Airport',
 'Travel']

In [236]:
tweets_df['Auto Context Annotations'][3]

[{'domain': {'id': '47',
   'name': 'Brand',
   'description': 'Brands and Companies'},
  'entity': {'id': '968089839210872833',
   'name': 'Heathrow Airport',
   'description': 'Heathrow Airport'}},
 {'domain': {'id': '65',
   'name': 'Interests and Hobbies Vertical',
   'description': 'Top level interests and hobbies groupings, like Food or Travel'},
  'entity': {'id': '839159814991167489',
   'name': 'Travel',
   'description': 'Travel'}}]

In [234]:
tweets_df['Entities'][0]

{'mentions': [{'start': 0,
   'end': 11,
   'username': 'HuygensKen',
   'id': '3290378591'}]}

In [231]:
user_geo

{'users': {<User id=540672851 name=Laz, Lazarou Monkiest of Terrors 3.5%🏳️‍🌈💙🚀 username=FrancisMaudeAdv>,
  <User id=1187038420645924866 name=George Nugent Travels username=GeorgeNTravels>,
  <User id=2198370810 name=Jane Braybrook 🦊🌳🌱 username=JaneBraybrook1>,
  <User id=848996189584449537 name=Margaret username=Margaretb2107>,
  <User id=103576640 name=Carol H Scott username=MsCarolHScott>,
  <User id=1283540598427983872 name=Richard Blaber username=RMBlaber56>,
  <User id=1402276468013608963 name=Robert James 7.0👌🇬🇧🇬🇬⚒️👌 username=RobertJ52942435>,
  <User id=88545686 name=Pop Quiz! 👻🕸️📝 username=SeriousBismuth>,
  <User id=1422975858797465603 name=moiibnkknub username=moibnkjaaaa>}}

In [217]:
users = {u for u in query.includes["users"]}
#places = {p["id"]: p for p in query.includes["places"]}
users

{<User id=1131876443539902464 name=ขนิฏฐา username=KiN49NMOQOdEyws>,
 <User id=228385326 name=WORLIFTS                             #thinksafety username=Lifting_Equip>,
 <User id=1300924825905451008 name=หมดศรัทธา username=PCock45555111>,
 <User id=284162333 name=Rob Horgan username=Robbiehorgs>,
 <User id=377957668 name=Sarah Burgess username=SarahABurgess>,
 <User id=1411965418298920964 name=Yet Another Ban 2 username=YetAnotherBan21>,
 <User id=391081429 name=alicia pivaro username=aliciapivaro>,
 <User id=1231173747308138496 name=bryan mclean username=bryanmc75292461>,
 <User id=804162745 name=Frances Cutler username=frances_cutler>,
 <User id=3363298269 name=viv holliday ✉ username=holliday_viv>}

In [207]:
query.includes["users"]

[<User id=284162333 name=Rob Horgan username=Robbiehorgs>,
 <User id=3363298269 name=viv holliday ✉ username=holliday_viv>,
 <User id=391081429 name=alicia pivaro username=aliciapivaro>,
 <User id=1300924825905451008 name=หมดศรัทธา username=PCock45555111>,
 <User id=228385326 name=WORLIFTS                             #thinksafety username=Lifting_Equip>,
 <User id=804162745 name=Frances Cutler username=frances_cutler>,
 <User id=377957668 name=Sarah Burgess username=SarahABurgess>,
 <User id=1231173747308138496 name=bryan mclean username=bryanmc75292461>,
 <User id=1131876443539902464 name=ขนิฏฐา username=KiN49NMOQOdEyws>,
 <User id=1411965418298920964 name=Yet Another Ban 2 username=YetAnotherBan21>]

In [195]:
for tweet in query.data:
    if users[tweet.author_id]:
        print(tweet.author_id)
        user = users[tweet.author_id]
        print(user.location)

284162333
London
3363298269
Can sometimes be found indoors. Yorkshire
391081429
London
1300924825905451008
Thailand
228385326
Wildmoor, Worcs.  B61 0QU
804162745
None
377957668
sotogrande, spain
1231173747308138496
None
1131876443539902464
จ.นนทบุรี, ประเทศไทย
1411965418298920964
None


# Resources
https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9